In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import datetime as dt
import matplotlib.pyplot as plt

# Configuración de semillas para reproducibilidad
SEED = 7
tf.keras.utils.set_random_seed(SEED)

# Configuración de GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

def preparar_datos(tamanio_ventana, horizonte, data):
    """Prepara los datos para entrenamiento"""
    n_features = len(data.columns)
    n_samples = (len(data) - tamanio_ventana - horizonte + 1) // horizonte + 1
    
    x_train = np.zeros((n_samples, tamanio_ventana, n_features))
    y_train = np.zeros((n_samples, horizonte))
    
    for i in range(tamanio_ventana, len(data) - horizonte + 1, horizonte):
        idx = (i - tamanio_ventana) // horizonte
        x_train[idx, :, :] = data.iloc[(i - tamanio_ventana):i, :]
        y_train[idx, :] = data['SIN'].iloc[i:(i + horizonte)]
    
    return x_train, y_train

def create_model(tamanio_ventana, cantidad_unidades, nfeatures, dropout):
    """Crea el modelo BiGRU"""
    model = keras.Sequential([
        keras.layers.Bidirectional(
            keras.layers.GRU(cantidad_unidades),
            input_shape=(tamanio_ventana, nfeatures)
        ),
        keras.layers.Dropout(rate=dropout),
        keras.layers.Dense(24)
    ])
    return model

def lr_step_decay(epoch, lr):
    """Programa de decay para learning rate"""
    initial_learning_rate = 5e-4
    drop = 0.5
    epochs_drop = 10
    return initial_learning_rate * (drop ** (epoch // epochs_drop))

# Cargar y preparar datos
df = pd.read_csv("processed_dataset.csv", index_col="Date")
df.index = pd.to_datetime(df.index)

# Crear features temporales
df["Month_sin"] = np.sin(2 * np.pi * df.index.month / 12)
df["Month_cos"] = np.cos(2 * np.pi * df.index.month / 12)
df["Year"] = df.index.year

# Seleccionar features
features = ["SIN", "Temperature", "Humidity", "Month_sin", "Month_cos", "Year"]
datos_norm = df[features]

# Configuración de parámetros
hist_window = 336
num_units = 48
batch_size = 16
learning_rate = 5e-4
dropout = 0.2
epochs = 150
pred_window = 24

# Preparar datos
scaler = MinMaxScaler()
scaler.fit(datos_norm.loc['2009-01-01':'2022-12-31'])

# Datos de entrenamiento
train = datos_norm.loc['2009-01-01':'2019-12-31']
train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns, index=train.index)
x_train, y_train = preparar_datos(hist_window, pred_window, train_scaled)

# Datos de validación
val = datos_norm.loc['2020-01-01':'2022-12-31']
val_scaled = pd.DataFrame(scaler.transform(val), columns=val.columns, index=val.index)
x_val, y_val = preparar_datos(hist_window, pred_window, val_scaled)

# Crear y entrenar modelo
model = create_model(hist_window, num_units, len(features), dropout)

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    keras.callbacks.LearningRateScheduler(lr_step_decay)
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mae",  # Mean Absolute Error
    metrics=[keras.metrics.RootMeanSquaredError()]
)

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=1,
    shuffle=False
)

# Predicciones y evaluación
predictions = model.predict(x_val)

# Revertir scaling para evaluación
predictions_original = scaler.inverse_transform(
    np.hstack((predictions.reshape(-1, 1), np.zeros((predictions.size, len(features) - 1))))
)[:, 0]

y_validation_original = scaler.inverse_transform(
    np.hstack((y_val.reshape(-1, 1), np.zeros((y_val.size, len(features) - 1))))
)[:, 0]

# Métricas de evaluación
mae = mean_absolute_error(y_validation_original, predictions_original)
mse = mean_squared_error(y_validation_original, predictions_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_validation_original, predictions_original)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# Gráfico simple de comparación
plt.figure(figsize=(12, 6))
plt.plot(y_validation_original[:100], label='Real', color='red', alpha=0.7)
plt.plot(predictions_original[:100], label='Predicho', alpha=0.7)
plt.title('Comparación: Valores Reales vs Predichos')
plt.xlabel('Muestras')
plt.ylabel('SIN (MW)')
plt.legend()
plt.grid(True)
plt.show()

# Guardar modelo
model.save("modelo_simplificado.h5")
print("Modelo guardado como 'modelo_simplificado.h5'")

h:\Anaconda\envs\deepf\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


AttributeError: module 'keras._tf_keras.keras.losses' has no attribute 'mean_absolute_error'